In [ ]:
#importing cifar10 dataset that comes with the keras library
from keras.datasets import cifar10
import glob
from PIL import Image
import numpy as np

#function that will load non face cifar images and normalize them
def load_non_face_images():
    (_,_),(cifar10_images,_)=cifar10.load_data()
    cifar10_images = cifar10_images[:60]
    cifar10_images = cifar10_images.astype('float32')/255.0
    return cifar10_images

#function that will load face images and normalize them
def load_face_images(dir_name):
    output = []
    for i in glob.glob(dir_name + '/*'):
        image = Image.open(i).resize((32,32))
        img_data = np.asarray(image, dtype='float32')/255.0
        oid = np.transpose(img_data, (2,0,1))
        output.append(oid)
    return np.asarray(output)


    


In [ ]:
#a function to seperate face and non face data into train and test images
def seperate_data(face_images, non_face_images):
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    
    for i in range(10):
        test_images.append(face_images[i])
        test_labels.append([1,0])
        test_images.append(non_face_images[i])
        test_labels.append([0,1])
        
    for i in range(len(face_images)-10):
        train_images.append(face_images[i])
        train_labels.append([1,0])
        train_images.append(non_face_images[i])
        train_labels.append([0,1])
    return np.asarray(train_images), np.asarray(train_labels), np.asarray(test_images), np.asarray(test_labels)

#function to flatten an image array 
def reshape_image(input_array):
    output_array = []
    for image in input_array:
        output_array.append(image.reshape(-1))
    return np.asarray(output_array)


In [ ]:
#importing all necessary libraries
from keras.models import Sequential
from keras.layers import Reshape, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras import optimizers
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_learning_phase(1)
from freeze_graph import freeze_graph
import tensorflow as tf

#functions to define, train and evaluate model
def create_model():
    model = Sequential()
    model.add(Reshape((3,32,32), input_shape = (3072,)))
    model.add(Conv2D(32, (3,3), input_shape = (3,32,32), padding = 'same', activation ='relu', kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', kernel_constraint = maxnorm(3)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation = 'softmax'))
    return model

def train_eval_model(model, train_images, train_labels):
    epochs = 50
    opt = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    print(model.summary)
    model.fit(train_images, train_labels, validation_data = (train_images,train_labels), epochs = epochs, batch_size = 12)
    
    
    #creating a protobuf file for android import
    frozen_graph = freeze_graph(K.get_session(), output_names=[model.output.op.name])
    tf.train.write_graph(frozen_graph,'.','C:/Users/Shubham/PycharmProjects/facialrecognition/protobuffile.pb',as_text=False)
    print(model.input.op.name)
    print(model.output.op.name)

    scores = model.evaluate(train_images,train_labels, verbose = 0)
    print("accuracy: %.2f%%"%(scores[1]*100))
    return model

#A function to check our trained model
def predict_image(model, test_images, test_labels):
    for i in range(len(test_labels)):
        test_image = np.expand_dims(test_images[i], axis = 0)
        print('Predicted_label: {}, actual label: {}'.format(model.predict(test_image), test_labels[i]))
        



In [ ]:
#Run the code
def run():
    face_images = load_non_face_images()
    non_face_images = load_face_images('faceimages')

    train_images, train_labels, test_images, test_labels = seperate_data(face_images, non_face_images)
    
    train_images = reshape_image(train_images)
    test_images = reshape_image(test_images)

    model = create_model()
    
    model = train_eval_model(model, train_images, train_labels)

    predict_image(model, test_images, test_labels)
    


In [ ]:
run()